In [1]:
import pandas as pd

In [2]:
#df = pd.read_csv('CustomerSegmentation.csv' , index_col = 0 , encoding = 'Unicode_escape' )

In [3]:
!pip install fastai

In [4]:
 pip install psycopg2-binary

In [5]:
from fastai.collab import *
from fastai.tabular.all import *

In [6]:
import psycopg2
from psycopg2 import sql
from sqlalchemy import create_engine , text

In [7]:
dbname = "railway"
user = "postgres"
password = "*B1AcCDbDegA4DD5DfB*C-fG5a6-c1e6"
host = "roundhouse.proxy.rlwy.net"
port = "55650"

In [8]:
engine2 = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{dbname}')

In [9]:
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)


In [10]:

cur = conn.cursor()

In [11]:
table_name = "final_book_rating"
query = sql.SQL("SELECT * FROM {}").format(sql.Identifier(table_name))

In [12]:
# Execute the query
cur.execute(query)

In [13]:
# Fetch all rows
rows = cur.fetchall()

In [14]:
# Create a DataFrame from the fetched data
columns = [desc[0] for desc in cur.description]
df = pd.DataFrame(rows, columns=columns)

In [15]:
df.describe()

User-ID   Book-Rating
count   78060.000000  78060.000000
mean    29439.987253      7.542070
std     45804.096213      1.853798
min         8.000000      1.000000
25%     11676.000000      6.000000
50%     22296.000000      8.000000
75%     32773.000000      9.000000
max    278854.000000     10.000000

In [16]:
unique_counts = df.nunique()
print(unique_counts)

User-ID        12966
ISBN           51474
Book-Rating       10
dtype: int64


In [17]:
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78060 entries, 0 to 78059
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   User-ID      78060 non-null  int64 
 1   ISBN         78060 non-null  object
 2   Book-Rating  78060 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.8+ MB


In [18]:
df.head()

User-ID        ISBN  Book-Rating
0   276726  0155061224            5
1   276729  052165615X            3
2   276729  0521795028            6
3   276736  3257224281            8
4   276737  0600570967            6

600

In [19]:
dfm = df

In [20]:
dfm.head()

User-ID        ISBN  Book-Rating
0   276726  0155061224            5
1   276729  052165615X            3
2   276729  0521795028            6
3   276736  3257224281            8
4   276737  0600570967            6

In [97]:
dfm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78060 entries, 0 to 78059
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   User-ID      78060 non-null  int64 
 1   ISBN         78060 non-null  object
 2   Book-Rating  78060 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.8+ MB


In [21]:
unique_counts = df.nunique()
print(unique_counts)

User-ID        12966
ISBN           51474
Book-Rating       10
dtype: int64


In [123]:
unique_counts = dfm.nunique()
print(unique_counts)

User-ID        12966
ISBN           51474
Book-Rating       10
dtype: int64


In [22]:
dls = CollabDataLoaders.from_df(dfm, item_name='ISBN', user_name='User-ID', rating_name='Book-Rating' , bs=64 , valid_pct= 0.2)
dls.show_batch()

In [23]:
learner_inf = collab_learner(dls, y_range=(0.6, 10.4))
learner_inf.fit_one_cycle(16)

In [24]:
learner_inf.model

EmbeddingDotBias(
  (u_weight): Embedding(11411, 50)
  (i_weight): Embedding(51475, 50)
  (u_bias): Embedding(11411, 1)
  (i_bias): Embedding(51475, 1)
)

In [21]:
user_bias = learner_inf.model.i_bias.weight
item_bias = learner_inf.model.u_bias.weight
#idxs = item_bias.argsort(descending=True)[:5]
#[dls.classes['StockCode'][i] for i in idxs]

In [ ]:
#print(idxs)  #these are movie bias

In [101]:
sale = learner_inf.model.i_weight.weight

In [102]:
print(sale)

Parameter containing:
tensor([[-0.0037, -0.0139,  0.0051,  ...,  0.0171,  0.0042, -0.0174],
        [ 0.0105, -0.0099, -0.0117,  ...,  0.0015, -0.0055, -0.0066],
        [-0.0694,  0.0816, -0.0851,  ..., -0.0792, -0.0701,  0.0918],
        ...,
        [-0.0648, -0.0639, -0.0558,  ...,  0.0682, -0.0601,  0.0703],
        [-0.0431,  0.0524, -0.0496,  ..., -0.0435, -0.0510,  0.0531],
        [ 0.0817,  0.0863,  0.0811,  ..., -0.0891,  0.0758, -0.0775]],
       device='cuda:0', requires_grad=True)


NameError: name 'size' is not defined

In [103]:
user = learner_inf.model.u_weight.weight

In [104]:
print(user)

Parameter containing:
tensor([[-0.0109, -0.0125, -0.0050,  ...,  0.0056,  0.0016, -0.0147],
        [-0.1272,  0.1333, -0.1124,  ..., -0.1502,  0.1760,  0.0993],
        [ 0.0065,  0.0095,  0.0044,  ..., -0.0075,  0.0095, -0.0070],
        ...,
        [-0.2116, -0.2534, -0.2473,  ..., -0.1761,  0.2704, -0.2700],
        [-0.0746, -0.0746, -0.0726,  ...,  0.0846, -0.0580,  0.0689],
        [-0.0453, -0.1396,  0.1094,  ..., -0.1022, -0.1337, -0.1526]],
       device='cuda:0', requires_grad=True)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (11363x50 and 51475x50)

In [108]:
sale_trans = sale.t()

In [109]:
mult = torch.matmul( user , sale_trans)

In [120]:
user.size()

torch.Size([11363, 50])

In [119]:
sale.size()

torch.Size([51475, 50])

In [110]:
print(mult)

tensor([[ 6.7723e-04, -2.0245e-04,  2.3774e-03,  ...,  7.6113e-03,
         -2.7330e-03, -9.7104e-03],
        [-4.3206e-03, -2.9723e-03, -2.3140e-03,  ...,  2.4276e-02,
          3.3598e-02, -3.8431e-02],
        [-4.8823e-04,  1.0378e-04,  1.1843e-02,  ..., -6.8834e-03,
          3.6299e-03,  9.7408e-03],
        ...,
        [ 1.6407e-02,  6.5147e-03, -4.2832e-02,  ...,  1.1729e-01,
          1.2679e-02, -1.9112e-01],
        [ 3.1688e-03,  7.9137e-03,  1.0404e-02,  ...,  1.1083e-01,
         -1.0895e-03, -1.3351e-01],
        [ 6.9872e-03,  6.9010e-03,  3.3370e-02,  ..., -5.5126e-02,
          1.6050e-02,  7.4089e-02]], device='cuda:0', grad_fn=<MmBackward0>)


In [28]:
#mult = torch.matmul( user , sale_trans)

In [111]:
mult.size()

torch.Size([11363, 51475])

In [112]:
adding = add(mult , item_bias)

In [113]:
addingtra = adding.t()

In [114]:
adding2 = add(addingtra , user_bias)

In [115]:
final = adding2.t()

In [116]:
final.size()

torch.Size([11363, 51475])

In [35]:
print(final)

tensor([[ 4.3262e-03,  1.0224e-02,  9.1243e-02,  ...,  7.7869e-02,
         -4.6001e-02, -8.5815e-02],
        [ 9.2560e-03,  1.7382e-02,  9.6480e-02,  ...,  1.0446e-01,
          2.5754e-04, -1.0461e-01],
        [-1.7556e-02, -1.0186e-02,  7.9992e-02,  ...,  4.2658e-02,
         -6.0354e-02, -8.7080e-02],
        ...,
        [ 2.1225e-01,  2.0913e-01,  2.3823e-01,  ...,  3.7974e-01,
          1.6160e-01, -7.5033e-02],
        [ 6.9232e-02,  8.0755e-02,  1.6168e-01,  ...,  2.4350e-01,
          1.8057e-02, -1.4720e-01],
        [ 2.0338e-01,  2.1007e-01,  3.1498e-01,  ...,  2.0788e-01,
          1.6553e-01,  1.9073e-01]], device='cuda:0', grad_fn=<TBackward0>)


In [36]:
out, inds = torch.max(final,dim=1) #since dim =1 ; we will get max of each row

In [37]:
print(inds)

tensor([23799,  4800, 17078,  ..., 23799, 13264, 17078], device='cuda:0')


In [38]:
dfd = pd.DataFrame()

In [39]:
dfd['bestsale'] = 'yh'

In [40]:
dfd['customerid'] = '0'

In [41]:
dfd.head()

Empty DataFrame
Columns: [bestsale, customerid]
Index: []

In [43]:
for i in range(len(inds)) :
  dfd['bestsale'].loc[i] = dfm['ISBN'].iloc[int(inds[i])]

In [45]:
dfd1 = dfm['User-ID'].unique()

In [46]:
print(dfd1)

[276726 276729 276736 ...  43803  43806  43807]


In [47]:
for i in range(len(dfd1)) :
    dfd['customerid'].loc[i] = dfd1[i]

In [48]:
dff = pd.DataFrame(columns=['customerid'])

In [49]:
dff = dff.rename(columns={'customerid': 'User-ID'})

In [50]:
for row in dfd1:
    dff = dff.append(pd.Series(row, index=dff.columns), ignore_index=True)

Streaming output truncated to the last 5000 lines.
<ipython-input-50-5c2903f30be7>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dff = dff.append(pd.Series(row, index=dff.columns), ignore_index=True)
<ipython-input-50-5c2903f30be7>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dff = dff.append(pd.Series(row, index=dff.columns), ignore_index=True)
<ipython-input-50-5c2903f30be7>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dff = dff.append(pd.Series(row, index=dff.columns), ignore_index=True)
<ipython-input-50-5c2903f30be7>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dff = dff.append(pd.Series(row, index=dff.columns), igno

In [51]:
dff.head()

User-ID
0  276726
1  276729
2  276736
3  276737
4  276744

In [52]:
dff.describe()

User-ID
count     12966
unique    12966
top      276726
freq          1

In [53]:
dff1 = pd.DataFrame(columns=['book'])

In [54]:
arr = np.empty(len(inds), dtype=object)

In [55]:
for i in range(len(inds)):
     arr[i] = dfm['ISBN'].iloc[int(inds[i])]

In [ ]:
#print(arr)

In [ ]:
#print(len(arr))

In [56]:
dff1 = pd.DataFrame({'book': arr})


In [57]:
dff1.head()

book
0  0812575350
1  0886775205
2  0385239165
3  0765302365
4  2743421444

In [58]:
dff1.describe()

book
count        11363
unique         572
top     0385239165
freq          2039

In [59]:
result = pd.concat([dff, dff1], axis=1 )

In [60]:
result.head()

User-ID        book
0  276726  0812575350
1  276729  0886775205
2  276736  0385239165
3  276737  0765302365
4  276744  2743421444

In [71]:
result.describe()

User-ID        book
count     12966       11363
unique    12966         572
top      276726  0385239165
freq          1        2039

In [61]:
# Get the indices of the second-largest elements along each row
index2, indices_second_largest = torch.topk(final, k=2, dim=1)

# Use the indices to get the second-largest elements
second_largest_elements = torch.gather(final, dim=1, index=indices_second_largest[:, 1].view(-1, 1))

In [70]:
len(index2)

11363

In [64]:
import torch

# Assuming indices_second_largest is your PyTorch tensor
indices_second_largest_cpu = indices_second_largest.cpu().detach().numpy()
second_largest_indices = indices_second_largest_cpu[:, 1]


In [65]:
print(second_largest_indices)

[13264  1493 23799 ...  8683  1521 23073]


In [69]:
len(second_largest_indices)

11363

In [68]:

# Convert the PyTorch tensor to a NumPy array
second_largest_elements_np = second_largest_elements.cpu().detach().numpy()

# Create a new DataFrame or add a new column to the existing DataFrame
result['2best'] = second_largest_elements_np.tolist()

ValueError: Length of values (11363) does not match length of index (12966)

In [ ]:
result['bestsale2nd'] = result['customerid'] + 5

In [ ]:
for i in range(len(inds)):
    result.loc[i, 'bestsale2nd'] = dfm['StockCode'].iloc[int(second_largest_indices[i])]

In [ ]:
result.head()

customerid stockcode bestsale2nd
0          4     22470       22196
1       2462     22470       22867
2      11112     22867       22470
3      12314     22470       22867
4      12341     22470       22867

In [ ]:
##THIRD LARGEST INDEXX

In [ ]:
# Get the indices of the second-largest elements along each row
index3, indices_third_largest = torch.topk(final, k=3, dim=1)

In [ ]:

# Convert the PyTorch tensor to a NumPy array
indices_third_largest_np = indices_third_largest.detach().numpy()
third_largest_indices = indices_third_largest_np[:, 2]

In [ ]:
print(third_largest_indices)

[ 59 274  12  12  12 274 372  12 372  59 372 274  77  12  12  59  59  59
  59  76  59 638  12 638  38 372 266 274 274  12 266  59  77  12  59 274
  76  59 274  59  76  77  77 266 372  76  12  77 266  12 372 372  12  76
  12  12 372  12  12  59 638 274 274  12]


In [ ]:
result['bestsale3rd'] = result['customerid'] + 5

In [ ]:
for i in range(len(inds)):
    result.loc[i, 'bestsale3rd'] = dfm['StockCode'].iloc[int(third_largest_indices[i])]

In [ ]:
result.head()

customerid stockcode bestsale2nd bestsale3rd
0          4     22470       22196       22867
1       2462     22470       22867       21544
2      11112     22867       22470       22196
3      12314     22470       22867       22196
4      12341     22470       22867       22196

In [ ]:
result.head()

customerid stockcode bestsale2nd bestsale3rd
0          4     22470       22196       22867
1       2462     22470       22867       21544
2      11112     22867       22470       22196
3      12314     22470       22867       22196
4      12341     22470       22867       22196

In [ ]:
result = result.dropna()

In [ ]:
table_name2 = "customerrecostock"
query1 = sql.SQL("TRUNCATE TABLE {}").format(sql.Identifier(table_name2))

In [ ]:
# Execute the query
cur.execute(query1)

In [ ]:
query2 = sql.SQL("COMMIT")
# Execute the query
cur.execute(query2)

In [ ]:
result.to_sql(table_name2, engine2, index=False, if_exists='append')

64

In [ ]:
#result.to_csv('finaloutput_recommender2.csv')